In [50]:
import pandas as pd
import helpers

from IPython.display import display
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
# from sklearn.metrics import log_loss
# from sklearn.naive_bayes import BernoulliNB
# from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
import numpy as np

# convert the Dates column of our provided data from string to datetime format.
train=pd.read_csv('train.csv', parse_dates = ['Dates'])
test=pd.read_csv('test.csv', parse_dates = ['Dates'])
training, validation = train_test_split(train, train_size=.60)

# Print the first 5 rows of the dataframe.
print train.head(2)


                Dates        Category                  Descript  DayOfWeek  \
0 2015-05-13 23:53:00        WARRANTS            WARRANT ARREST  Wednesday   
1 2015-05-13 23:53:00  OTHER OFFENSES  TRAFFIC VIOLATION ARREST  Wednesday   

  PdDistrict      Resolution             Address           X          Y  
0   NORTHERN  ARREST, BOOKED  OAK ST / LAGUNA ST -122.425892  37.774599  
1   NORTHERN  ARREST, BOOKED  OAK ST / LAGUNA ST -122.425892  37.774599  


In [51]:
def prep(data):
    days = pd.get_dummies(data.DayOfWeek)
    district = pd.get_dummies(data.PdDistrict)
    
    # Gets the hour portion form the "Dates" column
    hour = data.Dates.dt.hour
    
    #Build new array with binary info on location, day and hour and numbered crime
    train_data = pd.concat([hour, days, district], axis=1)
    return train_data


train_data = prep(training)
# 878049 x 18 columns
display(train_data.head())

,Dates,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
554939,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
854069,11,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
770680,22,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
512292,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
738457,23,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [52]:
#Convert crime labels to numbers
le_crime = preprocessing.LabelEncoder()
encodedCrime = le_crime.fit_transform(training.Category)
humanReadableCrime = le_crime.inverse_transform(encodedCrime)
print humanReadableCrime
print encodedCrime

['LARCENY/THEFT' 'DRUG/NARCOTIC' 'VANDALISM' ..., 'BURGLARY' 'NON-CRIMINAL'
 'OTHER OFFENSES']
[16  7 35 ...,  4 20 21]


In [53]:
from sklearn import tree
X = train_data
Y = encodedCrime
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [54]:
validation_data = prep(validation)
validationPredictions = clf.predict(validation_data)

In [55]:
humanReadableValidationPredictions = le_crime.inverse_transform(validationPredictions)

In [56]:
validationTrueLabels = le_crime.fit_transform(validation.Category)
display(validation.head())
print(le_crime.inverse_transform(validationPredictions[0:5]))

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
446186,2009-02-02 21:21:00,NON-CRIMINAL,COURTESY REPORT,Monday,MISSION,NONE,2000 Block of MISSION ST,-122.419520,37.764229
44882,2014-10-05 07:00:00,SEX OFFENSES FORCIBLE,SEXUAL BATTERY,Sunday,MISSION,NONE,500 Block of DOLORES ST,-122.426153,37.760619
716415,2005-03-22 17:35:00,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Tuesday,TENDERLOIN,PSYCHOPATHIC CASE,100 Block of GOLDEN GATE AV,-122.413048,37.781912
672122,2005-11-01 21:46:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Tuesday,TARAVAL,NONE,400 Block of PACHECO ST,-122.464815,37.750922
242674,2012-02-08 18:18:00,LARCENY/THEFT,PETTY THEFT SHOPLIFTING,Wednesday,TARAVAL,"ARREST, CITED",3200 Block of 20TH AV,-122.475647,37.728528


['LARCENY/THEFT' 'OTHER OFFENSES' 'DRUG/NARCOTIC' 'LARCENY/THEFT'
 'LARCENY/THEFT']


In [57]:
print validationPredictions[0:10]
print validationTrueLabels[0:10]
counter = 0
for i in range(len(validationPredictions)):
    if validationPredictions[i] == validationTrueLabels[i]:
        counter+=1

accuracy = counter/float(len(validationPredictions))
print accuracy

[16 21  7 16 16 16 21 21 16 21]
[20 28 20 32 16  4  7 19 21  1]
0.224263994078
